# 일급함수 디자인 패턴

구현 언어가 관련된 패턴을 결정함
일급함수 언어의 경우 패턴에 참여하는 일부 클래스의 객체를 함수로 교체하면 획일적인 코드의 반복을 줄일 수 있다!

## 전략 패턴의 리팩토링

### 고전구조
 - 콘텍스트(Order) -> 전략 (Promotion) - 알고리즘을 구현하는 컴포넌트의 인터페이스
 구체적인 전략 - 전략의 구상 서브클래스 중 하나
 
 전략 패턴 : 일련의 알고리즘을 정의하고, 각각을 하나의 클래스 안에 넣어 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.(전자상거래)
 
규칙: 충성포인트 1000점 이상 고객은 전체 주문에 5% 할인
      하나의 주문에서 20개 이상의 동일 상품을 구입 시 그 상품에 대해 10% 할인
      서로 다른 상품을 10종류 이상 주문하면 전체 주문에 7% 할인
      할인은 한번에 하나만 적용 가능 

In [4]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:  #it is context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
        
    def due(self):
        if self.promotion is None:
            discount = 0
        
        else:
            discount = self.promotion.discount(self)
        
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): #Stretagy: 추상 디바이스 -> ABC 클래스를 상속하여 추상 베이스 클래스를 선언할 수 있음
    
    @abstractmethod
    def discount(self, order):
        '''Return detailed prices of discount'''

class FidelityPromo(Promotion): #1st discount rule
    '''충성포인트 1000점 이상 고객은 전체 주문에 5% 할인'''
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion): #2nd discount rule
    '''하나의 주문에서 20개 이상의 동일 상품을 구입 시 그 상품에 대해 10% 할인'''
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >=20:
                discount += item.total() *0.1
        
        return discount
    
class LargeOrderPromo(Promotion): #3rd discount rule
    '''서로 다른 상품을 10종류 이상 주문하면 전체 주문에 7% 할인'''
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >=10:
            return order.total() *0.07
        
        return 0  
    

In [5]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5), 
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)
       ]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [6]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [7]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [8]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [9]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

### 함수지향 전략

리팩토링 - 함수로
동일전략 객체를 반복해서 사용할 경우 플라이웨이트를 공유하는 것이 좋음 -> 소스코드의 행 수, 유지보수 비용이 늘어남

In [9]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:  #it is context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
        
    def due(self):
        if self.promotion is None:
            discount = 0
        
        else:
            discount = self.promotion(self)
        
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

def fidelity_promo(order): #1st discount rule
    '''충성포인트 1000점 이상 고객은 전체 주문에 5% 할인'''
    
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order): #2nd discount rule
    '''하나의 주문에서 20개 이상의 동일 상품을 구입 시 그 상품에 대해 10% 할인'''
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() *0.1
        
    return discount
    
def large_order_promo(order): #3rd discount rule
    '''서로 다른 상품을 10종류 이상 주문하면 전체 주문에 7% 할인'''
        
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >=10:
        return order.total() *0.07
        
    return 0 

promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    '''return max discount'''
    return max(promo(order) for promo in promos) # 제너레이터표현식을 이용해서 promos 에 있는 각 함수를 order에 적용하고, 최대할인액 계산

In [3]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5), 
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [5]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [23]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### 단순한 접근법 -> 최선의 전략 선택하기

 - meta 접근 -> 그냥 리스트로 전략들 만들어서 넣으면 새 전략이 만들어질 때마다 추가해줘야 함
 - 버그 여지 있음
 
### 모듈에서 전략 찾기


In [10]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [11]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [13]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [ ]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:  #it is context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
        
    def due(self):
        if self.promotion is None:
            discount = 0
        
        else:
            discount = self.promotion(self)
        
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

def fidelity_promo(order): #1st discount rule
    '''충성포인트 1000점 이상 고객은 전체 주문에 5% 할인'''
    
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order): #2nd discount rule
    '''하나의 주문에서 20개 이상의 동일 상품을 구입 시 그 상품에 대해 10% 할인'''
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() *0.1
        
    return discount
    
def large_order_promo(order): #3rd discount rule
    '''서로 다른 상품을 10종류 이상 주문하면 전체 주문에 7% 할인'''
        
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >=10:
        return order.total() *0.07
        
    return 0 

promos = [globals()[name] for name in globals() #globals 함수가 반환된 딕셔너리에서 name 을 반복함
           if name.endswith('_promo')
            and name != 'best_promo']

def best_promo(order):
    '''return max discount'''
    return max(promo(order) for promo in promos) # 제너레이터표현식을 이용해서 promos 에 있는 각 함수를 order에 적용하고, 최대할인액 계산

In [ ]:
from collections import namedtuple
import inspect

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:  #it is context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            return self.__total
        
    def due(self):
        if self.promotion is None:
            discount = 0
        
        else:
            discount = self.promotion(self)
        
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

def fidelity_promo(order): #1st discount rule
    '''충성포인트 1000점 이상 고객은 전체 주문에 5% 할인'''
    
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order): #2nd discount rule
    '''하나의 주문에서 20개 이상의 동일 상품을 구입 시 그 상품에 대해 10% 할인'''
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() *0.1
        
    return discount
    
def large_order_promo(order): #3rd discount rule
    '''서로 다른 상품을 10종류 이상 주문하면 전체 주문에 7% 할인'''
        
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >=10:
        return order.total() *0.07
        
    return 0 

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)] #할인액을 계산하는 함수들만 모듈에 넣으면 작동함

def best_promo(order):
    '''return max discount'''
    return max(promo(order) for promo in promos) # 제너레이터표현식을 이용해서 promos 에 있는 각 함수를 order에 적용하고, 최대할인액 계산

## 명령

함수를 인수로 전달하면 명령디자인패턴도 구현을 단순하게 할 수 있음
목적
연산을 실행하는 객체(호출자Invoker)와 연산을 구현하는 객체(수신자Receiver) 를 분리하는 것
명령 객체를 수신자와 호출자 사이에 놓고, 명령은 execute() 로 인터페이스 구현 -> 수신자가 갖고 있는 메서드 호출
 - 호출자는 수신자의 인터페이스를 알 필요가 없음
 - 명령의 서브클래스로 다른 수신자 추가 가능
 - 호출자에 Command 객체 대신 함수를 바로 지정 가능
 - MacroCommand 와 같은 콜러블 객체는 필요한 상태를 보관하여 __call__() 외의 메서드도 제공 가능함
 - 함수가 호출된 후의 상태를 내부에 보관하기 위해 클로저 사용가능

In [ ]:
class MacroCommand:
    '''명령 리스트를 실행하는 명령'''
    
    def __init__(self, commands):
        self.commands = list(commands)
        
    def __call__(self):  # callable 
        for command in self.commands:
            command()